In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import trim, regexp_replace, lower, explode, trim, split
from pyspark.sql.types import StructType, StructField, StringType

In [2]:
spark_session = SparkSession.builder\
.master("spark://192.168.2.97:7077") \
.appName("claude_carlsson_hdfs")\
.config("spark.dynamicAllocation.enabled", True)\
.config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
.config("spark.shuffle.service.enabled", True)\
.config("spark.dynamicAllocation.executorIdleTimeout","30s")\
.config("spark.cores.max", 5)\
.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/12 15:34:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/12 15:34:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/03/12 15:34:40 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/03/12 15:34:40 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/03/12 15:34:40 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [3]:
def parse_json_body_to_string(json_filename):
    df = spark_session.read.json(json_filename)

    my_df_lower = df.select(lower("body").alias("lowercase_body"))

    charachters_to_delete = "[\.,\[\]\(\):_\-!?\'\+=;/&{}@$#*\"\\\\%><|~¨´¤]"

    my_df_clean = my_df_lower.withColumn("lowercase_body", regexp_replace(my_df_lower.lowercase_body, charachters_to_delete, ""))

    my_df_words = my_df_clean.select(explode(split("lowercase_body", "\s+")).alias("word"))

    return my_df_words

In [4]:
df_all_years = parse_json_body_to_string("hdfs://192.168.2.97:50000/user/ubuntu/RC_2008-01.json")

In [5]:
df_all_years.show(100)

+-------------+
|         word|
+-------------+
|          wow|
|        youre|
|            a|
|     buzzkill|
|         time|
|           to|
|         hang|
|          out|
|         with|
|            a|
|    different|
|        crowd|
|        there|
|    1smartass|
|             |
|          you|
|          can|
|        march|
|          for|
|           me|
|           im|
|        going|
|           to|
|            a|
|        party|
|          and|
|       having|
|         some|
|          fun|
|      deleted|
|      deleted|
|      deleted|
|         damn|
|           im|
|       really|
|        sorry|
|        about|
|         that|
|    sometimes|
|        there|
|         just|
|         isnt|
|       really|
|     anything|
|          one|
|          can|
|          say|
|           to|
|         make|
|       things|
|          any|
|       better|
|          but|
|           my|
|     thoughts|
|          are|
|         with|
|          you|
|         wait|
|       

In [6]:
spark_session.stop()